# Run allですべてのデータを準備する

* URL: https://ftp.ebi.ac.uk/pub/databases/impc/all-data-releases/

In [ ]:
RELEASE = 23.0

columns = ["marker_symbol", "marker_accession_id", "mp_term_name", "mp_term_id", "p_value", "effect_size",
           "female_ko_effect_p_value", "male_ko_effect_p_value", "female_ko_parameter_estimate","sex_effect_p_value", "male_ko_parameter_estimate", # sex differences
           "genotype_effect_p_value", "genotype_effect_parameter_estimate",
           "zygosity", # zygosity
           "pipeline_name", "procedure_name", # life-stage
           "allele_symbol", # map to Phendigm
           ]


## 1. Download IMPC dataset

In [ ]:
P = print
from pprint import pprint as PP
from collections import Counter as C
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import numpy as np
import pandas as pd
import polars as pl
import shutil
import pickle
import json
import gzip
import networkx as nx
import urllib.request
from tqdm import tqdm
import hashlib


In [ ]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

In [ ]:
%%bash
pwd

In [ ]:
# Phenodigm dataが存在していない場合には、ダウンロードを促す

if not Path("data", "phenodigm", "impc_phenodigm.csv").exists():
    raise FileNotFoundError("Please download impc phenodigm data from https://diseasemodels.research.its.qmul.ac.uk/.")

In [ ]:
# パスの設定
data_dir = Path("data/impc")
data_dir.mkdir(parents=True, exist_ok=True)
csv_path = data_dir / f"statistical-results-ALL-{RELEASE}.csv"

# ファイルが存在しない場合にダウンロードして解凍
if not csv_path.exists():
    # ダウンロード URL
    url = f"https://ftp.ebi.ac.uk/pub/databases/impc/all-data-releases/release-{RELEASE}/results/statistical-results-ALL.csv.gz"

    print(f"Downloading and extracting: {url}")

    # URL からファイルサイズ取得（tqdmのため）
    with urllib.request.urlopen(url) as response:
        total_size = int(response.info().get("Content-Length", -1))
        with tqdm.wrapattr(response, "read", total=total_size, desc="Downloading", unit="B", unit_scale=True) as r:
            with gzip.GzipFile(fileobj=r) as uncompressed:
                with open(csv_path, 'wb') as out_file:
                    shutil.copyfileobj(uncompressed, out_file)

In [ ]:
%%bash

# wc -l data/impc/statistical-results*.csv
# Release 22.1: 3165335
# Release 23.0: 2159931

# 1 min

## 2. Filter dataset by P value < 0.0001 (10^-4)


In [ ]:
path_statistical_all = Path("data", "impc", f"statistical-results-ALL-{RELEASE}.csv")
df_statistical_all = pd.read_csv(path_statistical_all)
df_statistical_all = df_statistical_all[columns]
# 30 seconds

In [ ]:
# print(len(data))
# Release 21.1: 2062772
# Release 22.0: 3165334
# Release 23.0: 2159930

In [ ]:
# Filter by p_value < 0.0001
threshold = 0.0001
filter_pvalue = df_statistical_all["p_value"] < threshold
filter_female_ko_pvalue = df_statistical_all["female_ko_effect_p_value"] < threshold
filter_male_ko_pvalue = df_statistical_all["male_ko_effect_p_value"] < threshold

df_statistical_filtered = df_statistical_all[filter_pvalue | filter_female_ko_pvalue | filter_male_ko_pvalue] #! 要確認！！！

# Filter by mp_term_id and mp_term_name are not NaN
df_statistical_filtered = df_statistical_filtered.dropna(subset=["mp_term_id"])
df_statistical_filtered = df_statistical_filtered.dropna(subset=["mp_term_name"])

# Filter by effect_size is not NaN
df_statistical_filtered = df_statistical_filtered.dropna(subset=["effect_size"])

In [ ]:
print(len(df_statistical_filtered))
# Release 22.0: 54059 rows
# Release 22.1: 54059 rows
# Release 23.0: 49299 rows

In [ ]:
df_statistical_filtered.to_csv(f"data/statistical_filtered-{RELEASE}.csv", index=False) # 2 sec

## Split data by mp_term_name

In [ ]:
df_statistical_filtered = pd.read_csv(f"data/statistical_filtered-{RELEASE}.csv")

In [ ]:
# data/mp_term_nameを作成

output_path = Path("data", "mp_term_name")
if output_path.exists():
    shutil.rmtree(output_path)
output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# 名前をクリーンにする関数を定義
def clean_name(name):
    return name.replace("/", "_").replace(" ", "_")

# mp_term_nameをクリーニングし、ユニークな値を取得
unique_mp_term_names = df_statistical_filtered['mp_term_name'].unique()

In [ ]:
# ユニークなmp_term_nameごとにフィルタリングしてCSVに保存: 5 sec
for mp_term_name in unique_mp_term_names:
    df_mp_term = df_statistical_filtered[df_statistical_filtered['mp_term_name'] == mp_term_name]
    clean_mp_term_name = clean_name(mp_term_name)
    df_mp_term.to_csv(f"data/mp_term_name/{clean_mp_term_name}.csv", index=False)
# 5 sec

## 3. TSUMUGIに必要なアノテーション情報を整理する

In [ ]:
df_statistical_filtered = pd.read_csv(f"data/statistical_filtered-{RELEASE}.csv")

### Annotate life stages

In [ ]:
# life_stageの初期割り当て
def assign_life_stage(pipeline_name):
    if pd.isna(pipeline_name):
        return "Early"
    if "Interval" in pipeline_name or "interval" in pipeline_name:
        return "Interval"
    elif "Late" in pipeline_name or "late" in pipeline_name:
        return "Late"
    else:
        return "Early"

df_statistical_filtered["life_stage"] = df_statistical_filtered["pipeline_name"].apply(assign_life_stage)

# Embryo 表現型に該当する procedure_name の一覧
embryo_phenotyping = [
    "Gross Morphology Embryo E9.5",
    "Viability E9.5 Secondary Screen",
    "OPT E9.5",
    "MicroCT E9.5",
    "Gross Morphology Placenta E9.5",
    "Gross Morphology Embryo E12.5",
    "Embryo LacZ",
    "Gross Morphology Placenta E12.5",
    "Viability E12.5 Secondary Screen",
    "Viability E14.5-E15.5 Secondary Screen",
    "Gross Morphology Placenta E14.5-E15.5",
    "MicroCT E14.5-E15.5",
    "Gross Morphology Embryo E14.5-E15.5",
    "Viability E18.5 Secondary Screen",
    "MicroCT E18.5",
    "Gross Morphology Embryo E18.5",
    "Gross Morphology Placenta E18.5"
]

# life_stageをEmbryoに上書き
df_statistical_filtered.loc[df_statistical_filtered["procedure_name"].isin(embryo_phenotyping), "life_stage"] = "Embryo"
df_annotated = df_statistical_filtered.reset_index(drop=True)

In [ ]:
print(len(df_annotated))
print(df_annotated["life_stage"].value_counts())
# 54059
# life_stage
# Early       45724
# Embryo       4253
# Late         4024
# Interval       58
# Name: count, dtype: int64

### Annotate Sex differences

In [ ]:
threshold = 0.0001

# 条件リスト
conditions = [
    (df_annotated["sex_effect_p_value"] < threshold) & (df_annotated["female_ko_effect_p_value"] < threshold) & (df_annotated["male_ko_effect_p_value"] > threshold),
    (df_annotated["sex_effect_p_value"] < threshold) & (df_annotated["male_ko_effect_p_value"] < threshold) & (df_annotated["female_ko_effect_p_value"] > threshold)
]

# 条件に対応する値
choices = ["female", "male"]

# np.selectで列を設定
df_annotated["sexdual_dimorphism"] = np.select(conditions, choices, default=None)
df_annotated = df_annotated.reset_index(drop=True)

# 結果を確認
print(RELEASE)
print(df_annotated["sexdual_dimorphism"].value_counts())

# RELEASE 22.1
# male      4915
# female    4146

# RELEASE 23.0
# male      5026
# female    4344

In [ ]:
# 確認
df_annotated.dropna(subset=["sexdual_dimorphism"])[["p_value", "sexdual_dimorphism", "effect_size"]].head(10)

### 遺伝型、性差、ライフステージのアノテーションを統合する

In [ ]:
print(df_annotated["zygosity"].value_counts())
# RELEASE 22.1
# zygosity
# homozygote      41444
# heterozygote    11921
# hemizygote        694

# RELEASE 23.0
# homozygote      48037
# heterozygote    14706
# hemizygote        902


In [ ]:
# colmuns = ["marker_symbol", "marker_accession_id", "mp_term_name", "mp_term_id", "p_value", "female_ko_effect_p_value", "male_ko_effect_p_value", "sexdual_dimorphism", "zygosity", "life_stage", "effect_size",]

# df_annotated = df_annotated[colmuns]

In [ ]:
# mp_term = "increased fasting circulating glucose level".strip()
# df_annotated[
#     (df_annotated["marker_symbol"] == "Dnase1l2") &
#     (df_annotated["mp_term_name"] == mp_term)
# ]

In [ ]:
# アノテーション列を追加（inplace）
def make_annotation(row) -> list[str]:
    # 遺伝型
    if row['zygosity'] == 'homozygote':
        annotate = "Homo"
    elif row['zygosity'] == 'heterozygote':
        annotate = "Hetero"
    else:
        annotate = "Hemi"

    # 性別
    if row['sexdual_dimorphism'] == "female":
        annotate += ", Female"
    elif row['sexdual_dimorphism'] == "male":
        annotate += ", Male"

    # life stage
    if row['life_stage'] in {"Embryo", "Early", "Interval", "Late"}:
        annotate += f", {row['life_stage']}"

    annotations = []

    annotations.append(f"{row['mp_term_name']} ({annotate})")

    return annotations

df_annotated["annotation"] = df_annotated.apply(make_annotation, axis=1)

df_exploded = df_annotated.explode("annotation").reset_index(drop=True)

# marker_symbol ごとに annotation をリスト化＆ソート
marker_annotation_map = (
    df_exploded
    .groupby("marker_symbol")["annotation"]
    .apply(lambda x: sorted(set(x)))
)

In [ ]:
# 例：Rhdの注釈を表示
print(marker_annotation_map["Rhd"])
# 例：Amtの注釈を表示 (Embryo)
print(marker_annotation_map["Amt"])
# 例：Spag4の注釈を表示 (重複が削除されているか)
print(marker_annotation_map["Spag4"])


In [ ]:
Path("data/annotation").mkdir(exist_ok=True, parents=True)
file_path = "data/annotation/symbol_mptermname.json"
marker_annotation_map.to_json(file_path, indent=4)

# json.dump(marker_annotation_map, open(file_path, "w"), indent=4, sort_keys=True)


In [ ]:
%%bash

grep -c "Male" data/annotation/symbol_mptermname.json | sed "s|^|Male: |"
grep -c "Female" data/annotation/symbol_mptermname.json | sed "s|^|Feale: |"

grep -c "Homo" data/annotation/symbol_mptermname.json | sed "s|^|Homo: |"
grep -c "Hetero" data/annotation/symbol_mptermname.json | sed "s|^|Hetero: |"
grep -c "Hemi" data/annotation/symbol_mptermname.json | sed "s|^|Hemi: |"

grep -c "Embryo" data/annotation/symbol_mptermname.json | sed "s|^|Embryo: |"
grep -c "Early" data/annotation/symbol_mptermname.json | sed "s|^|Early: |"
grep -c "Interval" data/annotation/symbol_mptermname.json | sed "s|^|Interval: |"
grep -c "Late" data/annotation/symbol_mptermname.json | sed "s|^|Late: |"

# RELEASE 22.1
# Male: 4915
# Feale: 4146
# Homo: 41444
# Hetero: 11921
# Hemi: 694
# Embryo: 4253
# Early: 45724
# Interval: 58
# Late: 4024

# RELEASE 23.0
# Male: 4480
# Feale: 3557
# Homo: 30977
# Hetero: 9625
# Hemi: 492
# Embryo: 4207
# Early: 34324
# Interval: 54
# Late: 2509

###  Phenodigmを用いたヒト疾患情報を取得する

In [ ]:
df_phenodigm = pd.read_csv(Path("data", "phenodigm", "impc_phenodigm.csv"))
P(len(df_phenodigm))

In [ ]:
# 各行について空白の数をカウント
space_counts = df_phenodigm['Mouse model description'].str.count(' ')

# 空白の数が2でない行を抽出（== split して3つにならない行）
invalid_rows = df_phenodigm[space_counts != 2]

# 結果表示
print(f"全体の件数: {len(df_phenodigm)}")
print(f"空白がちょうど2つでない行数: {len(invalid_rows)}")
print(invalid_rows.head())
# -> たった2つしかなく、`Phex<not yet available>`なので、この2つは無視する

In [ ]:
df_phenodigm = df_phenodigm[space_counts == 2]
P(len(df_phenodigm))

In [ ]:
df_phenodigm[['allele_symbol', 'zygosity', 'life_stage']] = df_phenodigm['Mouse model description'].str.split(' ', n=2, expand=True)
df_phenodigm = df_phenodigm.drop(columns=['Mouse model description'])

In [ ]:
P(df_phenodigm.columns)
P(df_phenodigm["allele_symbol"].head(3))
P(df_phenodigm["zygosity"].head(3))
P(df_phenodigm["life_stage"].head(3))

In [ ]:
# phenodigmの表記とimpcデータの表記を揃える

df_phenodigm = df_phenodigm.replace({'zygosity': {'hom': 'homozygote', 'het': 'heterozygote','hem': 'hemizygote'}})
df_phenodigm['life_stage'] = df_phenodigm['life_stage'].str.capitalize()
print(df_phenodigm["zygosity"].value_counts())
print(df_phenodigm["life_stage"].value_counts())

In [ ]:
df_annotated_phenodigm = \
    df_annotated.set_index(['allele_symbol','life_stage','zygosity']) \
    .join(df_phenodigm.set_index(['allele_symbol','life_stage','zygosity']), how='left', rsuffix='_phenodigm') \
    .reset_index()
print(len(df_annotated_phenodigm))

In [ ]:
columns_to_keep = [
    "marker_symbol", "Disorder name", "life_stage", "zygosity"
    ]
df_annotated_phenodigm = df_annotated_phenodigm[columns_to_keep].dropna(subset=["Disorder name"]).reset_index(drop=True)
df_annotated_phenodigm

In [ ]:
# アノテーション列を追加（inplace）
def make_annotation(row) -> list[str]:
    # 遺伝型
    if row['zygosity'] == 'homozygote':
        annotate = "Homo"
    elif row['zygosity'] == 'heterozygote':
        annotate = "Hetero"
    else:
        annotate = "Hemi"

    # life stage
    if row['life_stage'] in {"Embryo", "Early", "Interval", "Late"}:
        annotate += f", {row['life_stage']}"

    annotations = []

    annotations.append(f"{row['Disorder name']} ({annotate})")

    return annotations

df_annotated_phenodigm["annotation"] = df_annotated_phenodigm.apply(make_annotation, axis=1)

df_exploded = df_annotated_phenodigm.explode("annotation").reset_index(drop=True)

# marker_symbol ごとに annotation をリスト化＆ソート
marker_annotation_map = (
    df_exploded
    .groupby("marker_symbol")["annotation"]
    .apply(lambda x: sorted(set(x)))
)

In [ ]:
# 例：Phenodigmの注釈を表示 (Embryo)
print(marker_annotation_map["Arhgap31"])

In [ ]:
Path("data/annotation").mkdir(exist_ok=True, parents=True)
file_path = "data/annotation/symbol_disordername.json"
marker_annotation_map.to_json(file_path, indent=4)

### mp term nameとIMPCのPhenotype URLを紐付ける

In [ ]:
df_select = df_statistical_filtered[['mp_term_id', 'mp_term_name']].drop_duplicates()
# df_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
df_select

In [ ]:
dict_phenotype_url = dict()
for index, row in df_select.iterrows():
    mp_tern_id = row['mp_term_id']
    impc_url = f"https://www.mousephenotype.org/data/phenotypes/{mp_tern_id}"
    mp_term_name = row['mp_term_name']
    dict_phenotype_url[mp_term_name] = impc_url

print(dict_phenotype_url["small lymph nodes"])

In [ ]:
with open('data/annotation/mptermname_phenotypeurl.tsv', 'w') as f:
    for term, url in dict_phenotype_url.items():
        f.write(f"{term}\t{url}\n")

In [ ]:
%%bash

head -n 3 data/annotation/mptermname_phenotypeurl.tsv
wc -l data/annotation/mptermname_phenotypeurl.tsv
# Release 22.0: 664
# Release 23.0: 659

### marker symbolとMGI accession idを紐付ける

In [ ]:
df_select = df_statistical_filtered[['marker_symbol', 'marker_accession_id']].drop_duplicates()
# df_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
df_select
# Release 22.1: 7746 rows
# Release 23.0: 7934 rows

In [ ]:
dict_symbol_id = dict()
for index, row in df_select.iterrows():
    dict_symbol_id[row['marker_symbol']] = row['marker_accession_id']
print(dict_symbol_id["Ncam1"])

In [ ]:
json.dump(dict_symbol_id, open("data/annotation/symbol_mgiid.json", "w"), indent=4, sort_keys=True)
Path("data/annotation/symbol_mgiid.tsv").write_text("\n".join([f"{k}\t{v}" for k, v in dict_symbol_id.items()]))

In [ ]:
%%bash
head -n 3 data/annotation/symbol_mgiid.tsv

## 4. 表現型の類似度を求める

In [ ]:
file_path = Path("data", "annotation", "symbol_mptermname.json")

symbol_mptermname = json.load(open(file_path))
print(symbol_mptermname["Dpf2"])

In [ ]:
# 空リストを持つ要素を除外しつつ、値を list → set に変換して重複を削除する
symbol_mptermname = {k: set(v) for k, v in symbol_mptermname.items() if v}
print(symbol_mptermname["Dpf2"])

### Jaccard係数で集合の類似度を計算

In [ ]:

gene_pair_mp_similarity = []

for a, b in combinations(symbol_mptermname, 2):
    shared_mp = sorted(symbol_mptermname[a] & symbol_mptermname[b])
    shared_mp_number = len(shared_mp)
    union_mp_number = len(symbol_mptermname[a] | symbol_mptermname[b])
    overlap_ratio = shared_mp_number / union_mp_number

    gene_pair_mp_similarity.append([a, b, round(overlap_ratio, 3), shared_mp_number, shared_mp])

## 46s

In [ ]:
print(gene_pair_mp_similarity [:3])
print(len(gene_pair_mp_similarity))
# Release 22.0: 29996385
# Release 22.1: 29996385
# Release 23.0: 31470211

### 重複する表現型が閾値以上のものを抽出

In [ ]:
similarity_threshold = 0.2
num_shared_mp = 3

gene_pair_mp_similarity_filtered = []
for record in gene_pair_mp_similarity:
    if record[2] >= similarity_threshold or record[3] >= num_shared_mp:
        gene_pair_mp_similarity_filtered.append(record)

In [ ]:
print(gene_pair_mp_similarity_filtered[:3])
print(len(gene_pair_mp_similarity_filtered))
# Release 21.1: 134880
# Release 22.0: 133281 <- Homo/Hetero/Hemiおよび♂・♀の完全一致を考慮するようになったため、減少
# Release 22.1: 133281
# v0.3.0: 261216 <- Similarity_threshodのor条件をつけたため、増加
# Release 23.0 TSUMUGI v0.3.2: 241,645 (thres >= 0.5 or num >= 3)
# Release 23.0 TSUMUGI v0.3.2: 866,361 (thres >= 0.2 or num >= 3)


In [ ]:
Path("data", "overlap").mkdir(exist_ok=True, parents=True)
pickle.dump(gene_pair_mp_similarity, open("data/overlap/gene_pair_mp_similarity.pkl", "wb"))
pickle.dump(gene_pair_mp_similarity_filtered, open("data/overlap/gene_pair_mp_similarity_filtered.pkl", "wb"))

# 18 sec

### 生データをCSV形式で出力 （ダウンロード用）

In [ ]:
df_similarity = pd.DataFrame(gene_pair_mp_similarity)
df_similarity.columns = ["Gene1", "Gene2", "Jaccard Similarity", "Number of shared phenotype", "List of shared phenotypes"]
df_similarity.reindex(
    columns=["Gene1", "Gene2", "Number of shared phenotype", "Jaccard Similarity", "List of shared phenotypes"]
)
df_similarity["List of shared phenotypes"] = df_similarity["List of shared phenotypes"].apply(json.dumps)
df_similarity
# 30 sec

In [ ]:
output_path = Path("data/TSUMUGI_raw_data.csv.gz")

def get_head1000_hash(df: pd.DataFrame) -> str:
    # head(1000)だけを対象にハッシュ化
    csv_bytes = df.head(1000).to_csv(index=False, lineterminator='\n').encode('utf-8')
    return hashlib.md5(csv_bytes).hexdigest()

def file_head1000_hash(path: Path) -> str | None:
    if not path.exists():
        return None
    with gzip.open(path, "rt", encoding="utf-8") as f:
        lines = [next(f) for _ in range(1001)]  # 1行目がヘッダー
        csv_content = ''.join(lines).encode('utf-8')
        return hashlib.md5(csv_content).hexdigest()

# 比較
new_hash = get_head1000_hash(df_similarity)
existing_hash = file_head1000_hash(output_path)

if new_hash != existing_hash:
    df_similarity.to_csv(output_path, index=False, compression="gzip", lineterminator='\n')
    print("🔄 ファイルを更新しました") # 3 min
else:
    print("✅ 内容に変更がないためスキップしました")

In [ ]:
# df_overlap_filtered = pd.DataFrame(shared_ratios_filtered)
# df_overlap_filtered.columns = ["Gene1", "Gene2", "Jaccard Similarity", "Number of shared phenotype", "List of shared phenotypes"]
# df_overlap_filtered.reindex(
#     columns=["Gene1", "Gene2", "Number of shared phenotype", "Jaccard Similarity", "List of shared phenotypes"]
# )
# df_overlap_filtered["List of shared phenotypes"] = df_overlap_filtered["List of shared phenotypes"].apply(json.dumps)

# df_overlap_filtered.to_csv("data/TSUMUGI_filtered_data.csv.gz", index=False, compression="gzip", lineterminator='\n')

## 表現型ごとのネットワークを出力

In [ ]:
gene_pair_mp_similarity_filtered = pickle.load(open("data/overlap/gene_pair_mp_similarity_filtered.pkl", "rb"))

In [ ]:
df_similarity = pd.DataFrame(
    gene_pair_mp_similarity_filtered, columns=["marker1", "marker2", "phenotype_similarity", "shared_mp_number", "shared_mp"]
)
df_similarity
# version 0.2.2: 133281  rows × 5 columns
# version 0.3.0: 261216  rows × 5 columns

In [ ]:
marker_mp = json.load(open("data/annotation/symbol_mptermname.json"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term_name"])
marker_mp
# version 0.2.2: 7626 rows × 2 columns
# version 0.3.0: 7746 rows × 2 columns
# version 0.3.1: 7746 rows × 2 columns
# RELEASE 23.0: 7954 rows × 2 columns

In [ ]:
marker_disease = json.load(open("data/annotation/symbol_disordername.json"))

In [ ]:
output_dir = Path("data/network/mp_term_name")
# remove network directory
if output_dir.exists():
    shutil.rmtree(output_dir)

output_dir.mkdir(exist_ok=True, parents=True)


In [ ]:
path_mp_terms = list(Path("data", "mp_term_name").glob("*.csv"))
# print(path_mp_terms[:3])
# print(len(path_mp_terms))
# path_mp_term = Path("data", "mp_term_name", "increasing_circulating_glucose_level.csv")

"""
ノードが多すぎるとWebページが描画できない問題を回避するため、
ノード数を200以下にするために最適なphenotype_similarityを求める
"""
number_of_nodes = 200
tolerance = 25

for path_mp_term in path_mp_terms:

    df_marker_effect = pd.read_csv(path_mp_term).dropna(subset=["effect_size"])

    # Absolute value of effect size
    df_marker_effect["effect_size"] = df_marker_effect["effect_size"].abs()

    # * effect sizeの絶対値が最大の行を取得 (Homo/Heteroで異なる効果量がある場合に、ひとまず最大値を採用する← 今後の考慮事項)
    df_marker_effect = (
        df_marker_effect[["marker_symbol", "effect_size"]]  # 必要な列だけ抽出
        .loc[df_marker_effect.groupby("marker_symbol")["effect_size"].idxmax()]  # 各 marker_symbol ごとの effect_size 最大値の行を取得
        .reset_index(drop=True)
    )

    mp_term = path_mp_term.stem
    target_mp = mp_term.replace("_", " ")
    valid_markers = df_marker_effect['marker_symbol']

    df_filtered = df_similarity[
        df_similarity['marker1'].isin(valid_markers) &
        df_similarity['marker2'].isin(valid_markers) &
        df_similarity['shared_mp'].apply(lambda lst: any(target_mp in term for term in lst))
    ]

    # ----------------------------------------------------
    # Nodeを作成する
    # ----------------------------------------------------
    df_marker1 = df_filtered[["marker1"]]
    df_marker2 = df_filtered[["marker2"]]
    df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
    df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]

    df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()
    df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
    df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

    # ----------------------------------------------------
    # ノード数を200以下にするために最適なphenotype_similarityを求める
    # ----------------------------------------------------
    if len(df_node) > number_of_nodes:

        low, high = df_filtered["phenotype_similarity"].min(), df_filtered["phenotype_similarity"].max()

        best_phenotype_similarity = 0
        closest_diff = float('inf')  # 最も近いノード数の差を記録

        while low <= high:
            mid = (low + high) / 2

            df_mid = df_filtered[df_filtered["phenotype_similarity"] >= mid]

            # Nodeを作成する
            df_marker1 = df_mid[["marker1"]]
            df_marker2 = df_mid[["marker2"]]
            df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
            df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]
            df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()
            df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
            df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

            node_count = len(df_node)

            # ターゲットノード数に近い場合、best_phenotype_similarityを更新
            current_diff = abs(node_count - number_of_nodes)
            if current_diff < closest_diff:
                best_phenotype_similarity = mid
                closest_diff = current_diff 

            if number_of_nodes - tolerance < node_count < number_of_nodes + tolerance:
                break
            elif node_count > number_of_nodes:
                # ノード数が多い場合、範囲を上げる
                low = mid + 1e-6
            else:
                # ノード数が少ない場合、範囲を下げる
                high = mid - 1e-6

        df_filtered = df_filtered[df_filtered["phenotype_similarity"] >= best_phenotype_similarity]

    # NodeをJSON形式に変換
    node_json = []
    for _, row in df_node.iterrows():
        node_json.append({
            "data": {
                "id": row['marker_symbol'],
                "label": row['marker_symbol'],
                "phenotype": row['mp_term_name'],
                "disease": marker_disease[row['marker_symbol']] if row['marker_symbol'] in marker_disease else "",
                "node_color": row['effect_size']
            }
        })

    # ----------------------------------------------------
    # Edgeを作成する
    # ----------------------------------------------------
    df_edge = df_filtered[["marker1", "marker2", "phenotype_similarity", "shared_mp"]]
    # EdgeをJSON形式に変換
    edge_json = []
    for _, row in df_edge.iterrows():
        edge_json.append({
            "data": {
                "source": row['marker1'],
                "target": row['marker2'],
                "phenotype": row['shared_mp'],
                "edge_size": row['phenotype_similarity']
            }
        })

    # ----------------------------------------------------
    # EdgeとNodeを統合して、出力
    # ----------------------------------------------------

    network_json = node_json + edge_json

    # Output as JSON
    if network_json:
        output_json = output_dir / f"{mp_term}.json.gz"
        with gzip.open(output_json, "wt", encoding="utf-8") as f:
            json.dump(network_json, f, indent=4)

# 1m30s

In [ ]:
%%bash

ls -lhS data/network/mp_term_name/ | head -n 5

# version 0.2.2: total 5.3M
# version 0.3.0: total 5.5M
# version 0.3.1: total 5.1M <- 該当の表現型を含むネットワークのみを表示 （Issue: #54）
# version 0.3.2: total 5.1M <- Similarity 0.5 or 3 phenotypes
# version 0.3.3: total 9.8M <- Similarity 0.2 or 3 phenotypes

In [ ]:
%%bash
# 200前後のノード数となっているか確認

zcat data/network/mp_term_name/preweaning_lethality,_complete_penetrance.json.gz | grep -c "node_color"
# preweaning_lethality,_complete_penetranceは似ているノードが多すぎて、200を超えてしまう

zcat data/network/mp_term_name/edema.json.gz | grep -c "node_color"

In [ ]:

# path_mp_term = Path("data/mp_term_name/preweaning_lethality,_complete_penetrance.csv")

# number_of_nodes = 200
# tolerance = 25
# df_marker_effect = pd.read_csv(path_mp_term).dropna(subset=["effect_size"])

# # Absolute value of effect size
# df_marker_effect["effect_size"] = df_marker_effect["effect_size"].abs()

# # * effect sizeの絶対値が最大の行を取得 (Homo/Heteroで異なる効果量がある場合に、ひとまず最大値を採用する← 今後の考慮事項)
# df_marker_effect = (
#     df_marker_effect[["marker_symbol", "effect_size"]]  # 必要な列だけ抽出
#     .loc[df_marker_effect.groupby("marker_symbol")["effect_size"].idxmax()]  # 各 marker_symbol ごとの effect_size 最大値の行を取得
#     .reset_index(drop=True)
# )

# mp_term = path_mp_term.stem
# target_mp = mp_term.replace("_", " ")
# valid_markers = df_marker_effect['marker_symbol']

# df_filtered = df_similarity[
#     df_similarity['marker1'].isin(valid_markers) &
#     df_similarity['marker2'].isin(valid_markers) &
#     df_similarity['shared_mp'].apply(lambda lst: any(target_mp in term for term in lst))
# ]

# # ----------------------------------------------------
# # Nodeを作成する
# # ----------------------------------------------------
# df_marker1 = df_filtered[["marker1"]]
# df_marker2 = df_filtered[["marker2"]]
# df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
# df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]

# df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()
# df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
# df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

# best_phenotype_similarity = None
# best_under_node_count = -1
# best_under_mid = None

# print(len(df_node))
# # ----------------------------------------------------
# # ノード数を200以下にするために最適なphenotype_similarityを求める
# # ----------------------------------------------------
# if len(df_node) > number_of_nodes:

#     low, high = df_filtered["phenotype_similarity"].min(), df_filtered["phenotype_similarity"].max()
#     print(low, high)
#     while low <= high:
#         mid = (low + high) / 2

#         df_mid = df_filtered[df_filtered["phenotype_similarity"] >= mid]

#         # Nodeを作成する
#         df_marker1 = df_mid[["marker1"]]
#         df_marker2 = df_mid[["marker2"]]
#         df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
#         df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]
#         df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()
#         df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
#         df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

#         node_count = len(df_node)

#         if number_of_nodes - tolerance < node_count < number_of_nodes + tolerance:
#             best_phenotype_similarity = mid
#             break
#         elif node_count > number_of_nodes:
#             # ノード数が多い場合、範囲を上げる
#             low = mid + 1e-6
#         else:
#             # ノード数が少ない場合、範囲を下げる
#             best_under_node_count = node_count
#             best_under_mid = mid
#             high = mid - 1e-6

#     if best_phenotype_similarity is None:
#         best_phenotype_similarity = best_under_mid

#     df_optimized = df_filtered[df_filtered["phenotype_similarity"] >= best_phenotype_similarity]
#     G = nx.from_pandas_edgelist(df_optimized, "marker1", "marker2")
#     print(low, high)
#     print(len(G.nodes))

## 遺伝子ごとのネットワークを出力

In [ ]:
marker_mp_dict = dict(zip(marker_mp.marker_symbol, marker_mp.mp_term_name))

In [ ]:
gene_symbols = df_similarity.marker1.unique().tolist()
gene_symbols += df_similarity.marker2.unique().tolist()
gene_symbols = list(set(gene_symbols))
gene_symbols.sort()  # 以下のfor文で、どこまで遺伝子が処理されたのか途中経過を見積もるためのソート
P(gene_symbols[:3])
P(len(gene_symbols))  # 6003

In [ ]:
output_dir = Path("data", "network", "gene_symbol")
# remove network directory
if output_dir.exists():
    shutil.rmtree(output_dir)

output_dir.mkdir(exist_ok=True, parents=True)


In [ ]:
number_of_nodes = 200
tolerance = 25  # tolerance for the number of nodes

for i, gene_symbol in enumerate(gene_symbols):
    """
    ノードが多すぎるとWebページが描画できない問題を回避するため、
    ノード数を200以下にするために最適なphenotype_similarityを求める
    """
    if i % 100 == 0:
        print(f"Processing {int(i+1)}/{len(gene_symbols)}: {gene_symbol}")

    df_filtered = df_similarity[(df_similarity["marker1"] == gene_symbol) | (df_similarity["marker2"] == gene_symbol)]
    G = nx.from_pandas_edgelist(df_filtered, "marker1", "marker2")

    # ノードAと直接つながっているノードのみを取得
    neighbors = list(G.neighbors(gene_symbol))
    subgraph_nodes = [gene_symbol] + neighbors
    subgraph = G.subgraph(subgraph_nodes)

    if len(subgraph.nodes) > number_of_nodes:

        low, high = df_filtered["phenotype_similarity"].min(), df_filtered["phenotype_similarity"].max()

        best_phenotype_similarity = 0
        closest_diff = float('inf')  # 最も近いノード数の差を記録

        while low <= high:
            mid = (low + high) / 2

            # phenotype_similarity >= mid のデータをフィルタリング
            df_mid = df_filtered[df_filtered["phenotype_similarity"] >= mid]

            G = nx.from_pandas_edgelist(df_mid, "marker1", "marker2")
            # ノードAと直接つながっているノードのみを取得
            if gene_symbol in G:
                neighbors = list(G.neighbors(gene_symbol))
            else:
            # mid が大きすぎてノードが除外されすぎた（＝subgraph が小さくなりすぎた）と考えて、探索範囲の上限 (high) を下げる
                high = mid - 1e-6
                continue

            subgraph_nodes = [gene_symbol] + neighbors
            subgraph = G.subgraph(subgraph_nodes)

            node_count = len(subgraph.nodes)

            # ターゲットノード数に近い場合、best_phenotype_similarityを更新
            current_diff = abs(node_count - number_of_nodes)
            if current_diff < closest_diff:
                best_phenotype_similarity = mid
                closest_diff = current_diff 

            if number_of_nodes - tolerance < node_count < number_of_nodes + tolerance:
                break
            elif node_count > number_of_nodes:
                # ノード数が多い場合、範囲を上げる
                low = mid + 1e-6
            else:
                # ノード数が少ない場合、範囲を下げる
                high = mid - 1e-6

        # 最適なphenotype_similarityでフィルタリング
        df_optimized = df_filtered[df_filtered["phenotype_similarity"] >= best_phenotype_similarity]
        G = nx.from_pandas_edgelist(df_optimized, "marker1", "marker2")

        # ノードAと直接つながっているノードのみを取得
        neighbors = list(G.neighbors(gene_symbol))
        subgraph_nodes = [gene_symbol] + neighbors
        subgraph = G.subgraph(subgraph_nodes)
        
    # nodesを用意
    node_json = []
    for node in subgraph.nodes():
        annotation = marker_mp_dict[node]
        node_color = 1 if node == gene_symbol else 0
        node_json.append({
            "data": {
                "id": node,
                "label": node,
                "node_color": node_color,
                "phenotype": annotation,
                "disease": marker_disease[node] if node in marker_disease else "",
                }
            })

    # edgesを用意
    df_edge = df_similarity[
        (df_similarity["marker1"].isin(subgraph.nodes())) & (df_similarity["marker2"].isin(subgraph.nodes()))
    ]

    edge_json = []
    for edge in df_edge.itertuples():
        edge_json.append(
            {
                "data": {
                    "source": edge.marker1,
                    "target": edge.marker2,
                    "edge_size": edge.phenotype_similarity,
                    "phenotype": edge.shared_mp,
                }
            }
        )
    network_json = node_json + edge_json

    # Output as JSON
    if network_json:
        output_json = output_dir / f"{gene_symbol}.json.gz"
        with gzip.open(output_json, "wt", encoding="utf-8") as f:
            json.dump(network_json, f, indent=4)

# 10m

In [ ]:
%%bash
ls -lhS data/network/gene_symbol/ | head -n 5
# version 0.3.0: total 170M
# version 0.3.1: total 168M
# version 0.3.2: total 145M

# total 287M
# -rwxrwxrwx 1 kuno kuno 578K Jun 11 05:35 Ttbk2.json.gz
# -rwxrwxrwx 1 kuno kuno 558K Jun 11 05:22 Furin.json.gz
# -rwxrwxrwx 1 kuno kuno 558K Jun 11 05:35 Trpm7.json.gz
# -rwxrwxrwx 1 kuno kuno 555K Jun 11 05:29 Plpp3.json.gz

In [ ]:
%%bash
# 200前後のノード数となっているか確認
zcat data/network/gene_symbol/Crls1.json.gz | grep -c "node_color"

In [ ]:
# # ! DEBUG TEST
# number_of_nodes = 200
# tolerance = 25  # tolerance for the number of nodes
# gene_symbol = "Map2k1"
# print(gene_symbol)
# # 今の処理
# df_filtered = df_similarity[(df_similarity["marker1"] == gene_symbol) | (df_similarity["marker2"] == gene_symbol)]

# G = nx.from_pandas_edgelist(df_filtered, "marker1", "marker2")

# # ノードAと直接つながっているノードのみを取得
# neighbors = list(G.neighbors(gene_symbol))
# subgraph_nodes = [gene_symbol] + neighbors
# subgraph = G.subgraph(subgraph_nodes)

# P(len(subgraph.nodes))  # 200

# if len(subgraph.nodes) > number_of_nodes:
#     # 二分探索の範囲
#     low, high = df_filtered["phenotype_similarity"].min(), df_filtered["phenotype_similarity"].max()

#     # 最適なものが見つからなかった場合に備えて保持
#     best_under_node_count = -1
#     best_under_mid = 0
#     best_phenotype_similarity = 0
#     closest_diff = float('inf')  # 最も近いノード数の差を記録

#     while low <= high:
#         mid = (low + high) / 2

#         # phenotype_similarity >= mid のデータをフィルタリング
#         df_mid = df_filtered[df_filtered["phenotype_similarity"] >= mid]

#         G = nx.from_pandas_edgelist(df_mid, "marker1", "marker2")
#         # ノードAと直接つながっているノードのみを取得
#         if gene_symbol in G:
#             neighbors = list(G.neighbors(gene_symbol))
#         else:
#         # mid が大きすぎてノードが除外されすぎた（＝subgraph が小さくなりすぎた）と考えて、探索範囲の上限 (high) を下げる
#             high = mid - 1e-6
#             continue

#         subgraph_nodes = [gene_symbol] + neighbors
#         subgraph = G.subgraph(subgraph_nodes)

#         node_count = len(subgraph.nodes)

#         # ターゲットノード数に近い場合、best_phenotype_similarityを更新
#         current_diff = abs(node_count - number_of_nodes)
#         if current_diff < closest_diff:
#             best_phenotype_similarity = mid
#             closest_diff = current_diff 

#         if number_of_nodes - tolerance < node_count < number_of_nodes + tolerance:
#             break
#         elif node_count > number_of_nodes:
#             # ノード数が多い場合、範囲を上げる
#             low = mid + 1e-6
#         else:
#             # ノード数が少ない場合、候補として記録
#             if node_count > best_under_node_count:
#                 best_under_node_count = node_count
#                 best_under_mid = mid
#             # ノード数が少ない場合、範囲を下げる
#             high = mid - 1e-6

#     # 最適なphenotype_similarityでフィルタリング
#     df_optimized = df_filtered[df_filtered["phenotype_similarity"] >= best_phenotype_similarity]
#     G = nx.from_pandas_edgelist(df_optimized, "marker1", "marker2")

#     # ノードAと直接つながっているノードのみを取得
#     neighbors = list(G.neighbors(gene_symbol))
#     subgraph_nodes = [gene_symbol] + neighbors
#     subgraph = G.subgraph(subgraph_nodes)
    
# print(node_count, low, high, mid, best_phenotype_similarity, best_under_mid)
# P(len(subgraph.nodes))  # 200

In [ ]:
df_filtered["phenotype_similarity"].plot(kind="hist", bins=100, title=f"Phenotype Similarity Distribution for {gene_symbol}")

In [ ]:
Path("data/overlap/available_gene_symbols.txt").write_text("\n".join(gene_symbols) + "\n")
print(len(gene_symbols))  # 4416 -> 4244 → 6003 → 4139
# version 0.2.2: 4139
# version 0.3.0: 6812 (Life stageを考慮 + 類似度を追加)
# version 0.3.1: 6812
# version 0.3.2: 6904

In [ ]:
%%bash

uname -a # OS name
date +"%Y/%m/%d %H:%M:%S" # Last update